## Import libraries

In [1]:
import os
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import TensorBoard, EarlyStopping
import time

Using TensorFlow backend.


In [2]:
DATASET_PATH  = './data/whole_data_remove_outlier'
IMAGE_SIZE    = (224, 224)
NUM_CLASSES   = 2
BATCH_SIZE    = 32  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 172  # freeze the first this many layers for training
NUM_EPOCHS    = 50

WEIGHTS_FINAL = 'final-resnet50-final.h5'
TOP_LAYERS_CHECKPOINT_PATH = 'top-best.hdf5'
FINE_TUNED_CHECKPOINT_PATH = 'finetuned-best.hdf5'


## Define the training set and validation set

In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 19948 images belonging to 2 classes.
Found 4989 images belonging to 2 classes.


In [4]:
# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')


****************
Class #0 = cats
Class #1 = dogs
****************


## Define and compile the model

In [5]:
# create the base model
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = base_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
#x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(1024, activation='relu', name='relu')(x)
# add logistic layer
output_layer = Dense(2, activation='softmax', name='softmax')(x)
final_model = Model(input=base_model.input, output=output_layer)

D:\Anaconda2\envs\tensorflow-gpu\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
D:\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("so...)`
  if sys.path[0] == '':


## Set the checkpoint and train the model

In [10]:
if os.path.exists(TOP_LAYERS_CHECKPOINT_PATH):
    final_model.load_weights(TOP_LAYERS_CHECKPOINT_PATH)
    print("Checkpoint '" + TOP_LAYERS_CHECKPOINT_PATH + "' loaded.")

Checkpoint 'top-best.hdf5' loaded.


In [7]:
# Train only the top layers we added, these parameters are initialized randomly.
for layer in base_model.layers:
    layer.trainable = False

final_model.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Save the TensorBoard logs.# Save the model after every epoch.
mc_top = ModelCheckpoint(TOP_LAYERS_CHECKPOINT_PATH, monitor='val_acc',verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

logs = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6)

stp = EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='auto')


In [9]:
t0 = time.time()
# train the model
final_model.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        callbacks=[mc_top, reduce_lr, stp,logs])

t1 = time.time()
print('time consumed:', (t1-t0))

Epoch 1/50
623/623 [==============================] - 261s 419ms/step - loss: 0.1000 - acc: 0.9624 - val_loss: 0.0425 - val_acc: 0.9849
Epoch 2/50
623/623 [==============================] - 257s 413ms/step - loss: 0.0933 - acc: 0.9655 - val_loss: 0.0313 - val_acc: 0.9871
Epoch 3/50
623/623 [==============================] - 258s 413ms/step - loss: 0.0845 - acc: 0.9685 - val_loss: 0.0349 - val_acc: 0.9869
Epoch 4/50
623/623 [==============================] - 257s 412ms/step - loss: 0.0830 - acc: 0.9678 - val_loss: 0.0274 - val_acc: 0.9895
Epoch 5/50
623/623 [==============================] - 258s 414ms/step - loss: 0.0765 - acc: 0.9705 - val_loss: 0.0389 - val_acc: 0.9845
Epoch 6/50
623/623 [==============================] - 258s 415ms/step - loss: 0.0780 - acc: 0.9697 - val_loss: 0.0424 - val_acc: 0.9835
Epoch 7/50
623/623 [==============================] - 259s 415ms/step - loss: 0.0705 - acc: 0.9726 - val_loss: 0.0364 - val_acc: 0.9863
Epoch 8/50
623/623 [============================

In [11]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)
    


0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 pool1_pad
6 max_pooling2d_1
7 res2a_branch2a
8 bn2a_branch2a
9 activation_2
10 res2a_branch2b
11 bn2a_branch2b
12 activation_3
13 res2a_branch2c
14 res2a_branch1
15 bn2a_branch2c
16 bn2a_branch1
17 add_1
18 activation_4
19 res2b_branch2a
20 bn2b_branch2a
21 activation_5
22 res2b_branch2b
23 bn2b_branch2b
24 activation_6
25 res2b_branch2c
26 bn2b_branch2c
27 add_2
28 activation_7
29 res2c_branch2a
30 bn2c_branch2a
31 activation_8
32 res2c_branch2b
33 bn2c_branch2b
34 activation_9
35 res2c_branch2c
36 bn2c_branch2c
37 add_3
38 activation_10
39 res3a_branch2a
40 bn3a_branch2a
41 activation_11
42 res3a_branch2b
43 bn3a_branch2b
44 activation_12
45 res3a_branch2c
46 res3a_branch1
47 bn3a_branch2c
48 bn3a_branch1
49 add_4
50 activation_13
51 res3b_branch2a
52 bn3b_branch2a
53 activation_14
54 res3b_branch2b
55 bn3b_branch2b
56 activation_15
57 res3b_branch2c
58 bn3b_branch2c
59 add_5
60 activation_16
61 res3c_branch2a
62 bn3c_branch2a

## Release the parameters from additional layers

In [11]:
mc_fit = ModelCheckpoint(FINE_TUNED_CHECKPOINT_PATH, monitor='val_acc',
                          verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

#if os.path.exists(FINE_TUNED_CHECKPOINT_PATH):
#    final_model.load_weights(FINE_TUNED_CHECKPOINT_PATH)
#    print ("Checkpoint '" + FINE_TUNED_CHECKPOINT_PATH + "' loaded.")

In [12]:
for layer in final_model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in final_model.layers[FREEZE_LAYERS:]:
    layer.trainable = True
final_model.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
print(final_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [13]:
t2=time.time()

# train the model
final_model.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        callbacks=[mc_fit, reduce_lr, stp,logs])

t3 = time.time()
print('time consumed:', (t3-t2))

Epoch 1/50
623/623 [==============================] - 264s 424ms/step - loss: 0.0871 - acc: 0.9700 - val_loss: 0.0269 - val_acc: 0.9897
Epoch 2/50
623/623 [==============================] - 259s 416ms/step - loss: 0.0782 - acc: 0.9708 - val_loss: 0.0363 - val_acc: 0.9881
Epoch 3/50
623/623 [==============================] - 260s 417ms/step - loss: 0.0795 - acc: 0.9705 - val_loss: 0.0293 - val_acc: 0.9883
Epoch 4/50
623/623 [==============================] - 259s 415ms/step - loss: 0.0730 - acc: 0.9715 - val_loss: 0.0269 - val_acc: 0.9901
Epoch 5/50
623/623 [==============================] - 258s 415ms/step - loss: 0.0679 - acc: 0.9746 - val_loss: 0.0291 - val_acc: 0.9883
Epoch 6/50
623/623 [==============================] - 257s 413ms/step - loss: 0.0676 - acc: 0.9742 - val_loss: 0.0297 - val_acc: 0.9881
Epoch 7/50
623/623 [==============================] - 259s 416ms/step - loss: 0.0681 - acc: 0.9732 - val_loss: 0.0292 - val_acc: 0.9879
Epoch 8/50
623/623 [============================